In [27]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Function to read data from CSV
def load_data_from_csv(file_path):
    df = pd.read_csv(file_path, sep=',', header=0)
    df.columns = ['timestamp', 'value']
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

# Load actual data from CSV
data = load_data_from_csv('/Users/Dell/Downloads/rate_log.csv')

daily_report = {
    "Open": round(data["value"].iloc[0], 4),
    "Close": round(data["value"].iloc[-1], 4),
    "High": round(data["value"].max(), 4),
    "Low": round(data["value"].min(), 4),
    "Volatility": round(data["value"].std(), 4),
}

# Dash app setup
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("EUR/USD Exchange Rate Dashboard", style={"textAlign": "center"}),

    # Live Rate Display
    html.Div([
        html.H2("Current Exchange Rate", style={"textAlign": "center"}),
        html.Div(id="live-rate", style={"fontSize": "2rem", "textAlign": "center"}),
    ], style={"marginBottom": "20px"}),

    # Graph Section
    html.Div([
        dcc.Graph(id="rate-graph"),
    ]),

    # Daily Report Section
    html.Div([
        html.H3("Daily Report", style={"textAlign": "center"}),
        html.Table([
            html.Tr([html.Th("Metric"), html.Th("Value")]),
            html.Tr([html.Td("Open"), html.Td(daily_report["Open"])]),
            html.Tr([html.Td("Close"), html.Td(daily_report["Close"])]),
            html.Tr([html.Td("High"), html.Td(daily_report["High"])]),
            html.Tr([html.Td("Low"), html.Td(daily_report["Low"])]),
            html.Tr([html.Td("Volatility"), html.Td(daily_report["Volatility"])]),
        ], style={"margin": "0 auto", "width": "50%", "border": "1px solid black"}),
    ]),

    # Refresh Data Button
    html.Div([
        html.Button("Refresh Data", id="refresh-button", n_clicks=0, style={"margin": "20px auto", "display": "block"}),
    ]),
])

@app.callback(
    [Output("live-rate", "children"), Output("rate-graph", "figure")],
    [Input("refresh-button", "n_clicks")]
)
def update_dashboard(n_clicks):
    # Reload data from CSV
    updated_data = load_data_from_csv('/Users/Dell/Downloads/rate_log.csv')

    # Current rate display
    current_rate = f"{updated_data['value'].iloc[-1]:.4f} USD"

    # Time series graph
    fig = px.line(updated_data, x="timestamp", y="value", title="EUR/USD Exchange Rate Over Time")
    fig.update_layout(xaxis_title="Time", yaxis_title="Exchange Rate (USD)")

    return current_rate, fig

if __name__ == "__main__":
    print("Starting the dashboard. You can access it at http://127.0.0.1:8050/")
    app.run_server(debug=True)


Starting the dashboard. You can access it at http://127.0.0.1:8050/
